In [ ]:
#%load_ext autoreload
#%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
import hashlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import Video as JupyterVideo
from IPython.display import HTML
from IPython.display import display

%matplotlib widget
#%pylab inline

import datajoint as dj

from os import system, name ##For clearing output 

import pose_pipeline.pipeline
import pose_pipeline
from pose_pipeline.utils.jupyter import play
pose_pipeline.env.set_environmental_variables()

In [ ]:
filt = '(video_project="SPASTICITY" or video_project="PARKINSONS" or video_project="SLING" or video_project="AMPUTEE_SAMPLES" or video_project="PROSTHETIC_GAIT")' #' and tracking_method=0'

pose_pipeline.BlurredVideo.populate(filt)
pose_pipeline.TrackingBboxVideo.populate(filt)

In [ ]:
filt = 'video_project="SPASTICITY" or video_project="SLING" or video_project="AMPUTEE_SAMPLES" or video_project="PARKINSONS"'
filt = 'video_project="SPASTICITY"'
filt = 'video_project="PROSTHETIC_GAIT"'
#filt = 'video_project="PARKINSONS"'
#filt = 'video_project="PARKINSONS" and tracking_method=1'
to_label = ((pose_pipeline.TrackingBboxVideo & filt) - pose_pipeline.PersonBboxValid).fetch('KEY')

print(len(to_label))

def assign_video(labeling_key, valid, tracks=None):
    """
    Params:
        labeling_key: dictionary indicating the video being visualized to annotate
        valid: boolean indicating a valid person could be identified
        tracks: list matching the numbers associated with the person in video
    """
    
    # video_subject_id = 0 corresponds to a valid gait lab subject, and
    # video_subject_id = -1 corresponds to a video with bad person detection
    # the keep_tracks fields should correspond to the number(s) overlying the
    # person in the video

    video_key = (pose_pipeline.pipeline.TrackingBbox & labeling_key).fetch1('KEY')

  
    key = video_key.copy()
    if valid:
        key['video_subject_id'] = 0
        key['keep_tracks'] = tracks
    else:
        key['video_subject_id'] = -1
        key['keep_tracks'] = []
        
    print(f'Inserting {key} into database') #Inserts into database 
    pose_pipeline.pipeline.PersonBboxValid.insert1(key)
    
def unlabel(labeling_key):
    (pose_pipeline.PersonBboxValid & labeling_key).delete()


def show_annotation(idx):
    
    labeling_key = to_label[idx]
    print(f'Showing: {labeling_key}')
    
    video = (pose_pipeline.pipeline.TrackingBboxVideo & labeling_key).fetch1('output_video')
    
    # get the track IDs present in the video
    tracks = (pose_pipeline.pipeline.TrackingBbox & labeling_key).fetch1('tracks')
    tracks = np.unique([[t['track_id'] for track in tracks for t in track]]).tolist()
    
    display(JupyterVideo(video, width=320, html_attributes="controls muted autoplay"))
    practice = widgets.ToggleButtons(
        options= ['Default', *tracks, 'Multiple', 'Skip','Undo', 'Invalid'],
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Can't Identify Subject", '0', '1', '2', '3', 'Skip']
    )   
    display(practice)
    
    multiple = widgets.Text(
        value='',
        placeholder='',
        description='Multiple',
        disabled=False
    )
    display(multiple)

    def on_click(change):
        value = change['new']
        if value == 'Multiple':
            value = [int(v) for v in multiple.value.split(',')]
            assign_video(labeling_key, True, value)
        elif value == 'Skip':
            print('Skipping')
        elif value == 'Invalid':
            print('Flagging Invalid')
            assign_video(labeling_key, False)
        elif value == 'Undo':
            unlabel(to_label[idx-1])
            show_annotation(idx-1)
            return
        else:
            value = int(value)
            assign_video(labeling_key, True, [value])
       
        os.remove(video)
        
        show_annotation(idx + 1)

    practice.observe(on_click, 'value')
    
show_annotation(0)
